<a href="https://colab.research.google.com/github/rikurb8/workbench/blob/main/backend/dalle_playground_backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src="https://em-content.zobj.net/thumbs/240/apple/325/artist_1f9d1-200d-1f3a8.png" width="120">
</center>

### <center>Use this notebook to run your text-to-image server</center>
### <center> [DALL-E Playground Repository](https://github.com/saharmor/dalle-playground) </center>

## Install dependencies

In [4]:
!git clone https://github.com/saharmor/dalle-playground.git
!pip install -r dalle-playground/backend/requirements.txt
!pip install diffusers["torch"]

fatal: destination path 'dalle-playground' already exists and is not an empty directory.


# Run the backend web server

In [6]:
from threading import Thread, Event

app_port = 8000
announce_url = None
cloudflared_startup = Event()

def update_announce_url(url):
    global announce_url
    announce_url = url

def start_cloudflared(port):
  from flask_cloudflared import _run_cloudflared
  try:
    announce_url = _run_cloudflared(port, 8888)
  except:
    raise
  finally:
    update_announce_url(announce_url)
    cloudflared_startup.set()

def run_with_cloudflared(thread):
    old_run = thread.run

    def new_run(*args, **kwargs):
        new_thread = Thread(target = start_cloudflared, args=(app_port, ))
        new_thread.setDaemon(True)
        new_thread.start()
        old_run(*args, **kwargs)

    thread.run = new_run

def app():
  !python dalle-playground/backend/app.py --port {app_port} --save_to_disk true --img_format jpeg --output_dir generations

if __name__ == '__main__':
    t1 = Thread(target = app)
    run_with_cloudflared(t1)
    t1.start()
    cloudflared_startup.wait()
    print(f"Your url is: {announce_url}")
    t1.join()

<ipython-input-6-fdfb929e2844>:26: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  new_thread.setDaemon(True)


Your url is: https://pupils-indiana-spaces-pmid.trycloudflare.com
2024-10-21 20:49:33.283270: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-21 20:49:33.303892: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-21 20:49:33.310557: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-21 20:49:33.325842: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate com

### Now, take the url you got from cloudflare (should look like `your url is: https://xxxxxx.trycloudflare.com`), replace it within the url here https://saharmor.github.io/dalle-playground/?backendUrl=https://xxxxxx.trycloudflare.com and run it in the browser.

### Let the fun begin ✨